### Monitoring wetland dynamics and its impact on malaria in the Zambesi delta, Mozambique

#### Context
What are wetlands?, why important?  
Why monitor?  
Relationship to malaria? Why it matters?  

### Method Description
1. Tasselled-Cap Wetnees (TCW) from surface reflectance
2. Fractional Cover
3. Masking using WOfS and TCW
4. Stacking and classifying the dominant land type
5. Comparison to malaria incidence rate 

#### 1. Tasseled-Cap Wetness(TCW) computaion from surface reflectance
TCW is one of the Tasseled-Cap Transform (TCT) indices, which uses a linear combination of Landsat bands to detect moisture content in soil and vegetation. The equations vary by Landsat sensor:

Landsat 5 & 7 (TM) Coefficients  
TCW=0.1509B1+0.1973B2+0.3279B3+0.3406B4−0.7112B5−0.4572B7  
Landsat 7 (ETM+) Coefficients  
TCW=0.2626B1+0.2141B2+0.0926B3+0.0656B4−0.7629B5−0.5388B7  
Landsat 8 & 9 (OLI) Coefficients  
TCW=0.1511𝐵2+0.1973𝐵3+0.3283𝐵4+0.3407𝐵5−0.7117𝐵6−0.4559𝐵7  
TCW=0.1511B2+0.1973B3+0.3283B4+0.3407B5−0.7117B6−0.4559B7  

B1, B2, etc., are Landsat bands (Surface Reflectance)

TCW is sensitive to water content in soil and vegetation

Higher TCW means more moisture, lower TCW means drier surfaces

In [ ]:
import datacube
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

from deafrica_tools.datahandling import load_ard, mostcommon_crs, wofs_fuser
from deafrica_tools.plotting import rgb, display_map, plot_wofs
from datacube.utils import masking, geometry
from datacube.utils.geometry import CRS
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import HiddenPrints
from deafrica_tools.spatial import xr_rasterize


In [ ]:
#connect to datacube
dc = datacube.Datacube(app="Landsat_Surface_Reflectance")

In [ ]:
# List Landsat products available in DE Africa
dc_products = dc.list_products()
display_columns = ['name', 'description']
dc_products[dc_products.name.str.contains(
    'sr').fillna(
        False)][display_columns].set_index('name')

In [ ]:
dc_measurements = dc.list_measurements()
# Display available measurements for Landsat 8
landsat_collections = ["ls8_sr"] 
for collection in landsat_collections:
    collection in dc_measurements.index
    print(f"Measurements for {collection}:")
    display(dc_measurements.loc[collection])

In [ ]:
# Bounding box for Mopeia in Zambezi delta Mozambique
lat, lon = -17.97757, 35.7130
buffer = 0.177

# Add lat, lon, buffer together to get bounding box
x = (lon - buffer, lon + buffer)
y = (lat - buffer, lat + buffer)

print(x, y)

# View the location 
display_map(x=x, y=y)

In [ ]:
from shapely.geometry import box

# Create the bounding box geometry using Shapely's box
bbox = box(x[0], y[0], x[1], y[1])

# Create a GeoDataFrame with the bounding box geometry
gdf = gpd.GeoDataFrame({'geometry': [bbox]}, crs="EPSG:32636")

# View the GeoDataFrame
print(gdf.geometry)
print(gdf.crs)

# Convert GeoDataFrame to a Geometry object for the query
geom = geometry.Geometry(geom=gdf.iloc[0].geometry, crs=gdf.crs)
print(geom)

In [ ]:
# Define the function to create a query
def create_query(location, year):
    return {
        'geopolygon': location,  # location is the Geometry object
        'time': (f"{year}-01-01", f"{year}-12-31"),  # Time range format (YYYY-MM-DD)
    }

# Define the years to query
years = [2017, 2020, 2023]


In [ ]:
# Load ARD Landsat data for the specified time range (2013-2022) every 3rd year
# Initialize lists to store datasets for each product
ds_landsat_list = []

# Load products for each year (one by one, each product separately)
for year in years:
    print(f"Processing data for {year}...")
    
    # Define the query for the current year
    query = create_query(geom, year)
    
    ds_landsat = load_ard(
        dc=dc,
        products=["ls8_sr"], 
        output_crs="epsg:32636",
        min_gooddata=0.85,
        mask_filters=(["opening", 3], ["dilation", 3]),
        measurements=["red", "green", "blue", "nir", "swir_1", "swir_2"],
        dask_chunks={'time': -1, 'x': 250, 'y': 250},  # Adjust chunking as needed
        group_by="solar_day",
        resolution=(-30, 30),
        **query
    )
    
    # Append the dataset to the list
    ds_landsat_list.append(ds_landsat)

# Concatenate along the 'time' dimension
ds_ls = xr.concat(ds_landsat_list, dim='time')

# Sort by time just in case
ds_ls = ds_ls.sortby('time')

print(ds_ls)


In [ ]:
# Visual sanity check
first_timestep = ds_ls.swir_1.isel(time=1)

print(first_timestep)
first_timestep.plot(cmap='Blues_r')

In [ ]:
TCW_threshold = -0.035

# 1. Rasterize the mask (optional but good if you have a shapefile AOI)
mask = xr_rasterize(gdf.iloc[[0]], ds_ls)
ds_ls = ds_ls.where(mask)

# 2. Calculate TCW using DE Africa tools
print("Calculating tasseled cap wetness index...")

with HiddenPrints():  # suppress prints from calculate_indices
    tcw = calculate_indices(
        ds_ls, 
        index=["TCW"], 
        normalise=False, 
        satellite_mission="ls", 
        drop=True
    )

# 3. Optional: Resample (e.g., monthly)
if resample_frequency is not None:
    print("Resampling TCW to", resample_frequency)
    tcw = tcw.resample(time=resample_frequency).max()

# 4. Apply threshold to get binary wet/dry
tcw = tcw.TCW >= TCW_threshold

# 5. Mask non-AOI pixels again just in case
tcw = tcw.where(mask, 0)

# 6. Persist for efficient use later
tcw = tcw.persist()


In [ ]:
#calculate tasseled cap wetness (TCW)
def tasseled_cap_wetness(ds):
    coefficients = {
        'blue': 0.2630,
        'green': 0.1456,
        'red': 0.3379,
        'nir': 0.4556,
        'swir1': 0.6140,
        'swir2': 0.1126
    }
    
    wetness = (
        ds['blue'] * coefficients['blue'] +
        ds['green'] * coefficients['green'] +
        ds['red'] * coefficients['red'] +
        ds['nir'] * coefficients['nir'] +
        ds['swir_1'] * coefficients['swir1'] +
        ds['swir_2'] * coefficients['swir2']
    )
    
    return wetness
    
tcw = tasseled_cap_wetness(ds_ls)
print(tcw)

In [ ]:
tcw.isel(time=1).plot(cmap='Blues_r')  # Plot time step

#### 2. WOfS 

In [ ]:
dc_wofs = datacube.Datacube(app="Intro_WOfS")

In [ ]:
#load the wofs features
# Initialize lists to store datasets for each product
ds_wofs_list = []

# Load products for each year (one by one, each product separately)
for year in years:
    print(f"Processing data for {year}...")
    
    # Define query for this year
    query = create_query(location, year)
    
    ds_wofsls = dc_wofs.load(
        product="wofs_ls",
        like=ds_landsat,
        fuse_func=wofs_fuser,
        dask_chunks={}, #{'time':-1, 'x':250, 'y':250}
        collection_category="T1",
    )
    ds_wofs_list.append(ds_wofsls)  # Append WOfS data to the list

# Concatenate along the 'time' dimension
ds_wofls = xr.concat(ds_wofs_list, dim='time')

# Sort by time just in case
ds_wofls = ds_wofls.sortby('time')

print(ds_wofls)


In [ ]:
# maksing the wet and dry areas
verbose= True
resample_frequency= "1M"

# boolean of wet/dry
wofls_wet = masking.make_mask(ds_wofls.water, wet=True)

if resample_frequency is not None:
    if verbose:
        print("Resampling WOfS to " + resample_frequency)
    wofls_wet = wofls_wet.resample(time=resample_frequency).max()

# mask sure wofs matches other datasets
wofls_wet = wofls_wet.where(wofls_wet.time == tcw.time)

# apply the polygon mask
wofls_wet = wofls_wet.where(mask)

#### 3. Fractional Cover (FC)

In [ ]:
# load fractional cover
    fc_ds = dc.load(
        product="fc_ls",
        time=time,
        dask_chunks=dask_chunks,
        like=ds_ls,
        measurements=["pv", "npv", "bs"],
        collection_category="T1",
    )

#### 4. Stacking and classifying land type

#### 5. compare statistics: water inundation vs malaria incidence rate